### Notebook purpose:

Simulate the different figures given different parameters

In [2]:
# OLD CODE FROM nanotools_mumax3.ipynb notebook - standard mumax3 code

from pandas import read_table
from subprocess import run, PIPE, STDOUT
from glob import glob
import os
from numpy import load
from IPython.display import display, Image

# def read_mumax3_table(filename):
#     """Puts the mumax3 output table in a pandas dataframe"""

#     table = read_table(filename)
#     table.columns = ' '.join(table.columns).split()[1::2]

#     return table

# def read_mumax3_ovffiles(outputdir):
#     """Load all ovffiles in outputdir into a dictionary of numpy arrays
#     with the ovffilename (without extension) as key"""

#     # convert all ovf files in the output directory to numpy files
#     p = run(["mumax3-convert","-numpy",outputdir+"/*.ovf"], stdout=PIPE, stderr=STDOUT)
#     if p.returncode != 0:
#         print(p.stdout.decode('UTF-8'))

#     # read the numpy files (the converted ovf files)
#     fields = {}
#     for npyfile in glob(outputdir+"/*.npy"):
#         key = os.path.splitext(os.path.basename(npyfile))[0]
#         fields[key] = load(npyfile)

#     return fields

# def run_mumax3(script, name, verbose=False):
#     """ Executes a mumax3 script and convert ovf files to numpy files

#     Parameters
#     ----------
#       script:  string containing the mumax3 input script
#       name:    name of the simulation (this will be the name of the script and output dir)
#       verbose: print stdout of mumax3 when it is finished
#     """
#     if not os.path.exists("simulations"):
#         os.mkdir("simulations")

#     scriptfile = "simulations/" + name + ".txt"
#     outputdir  = "simulations/" + name + ".out"

#     # write the input script in scriptfile
#     with open(scriptfile, 'w' ) as f:
#         f.write(script)

#     # call mumax3 to execute this script
#     p = run(["mumax3","-f",scriptfile], stdout=PIPE, stderr=STDOUT)
#     if verbose or p.returncode != 0:
#         print(p.stdout.decode('UTF-8'))

#     if os.path.exists(outputdir + "/table.txt"):
#         table = read_mumax3_table(outputdir + "/table.txt")
#     else:
#         table = None

#     fields = read_mumax3_ovffiles(outputdir)

#     return table, fields

# def convert_ovf_to_png(simulation_name):
#     """Converts ovf to .png files within a given folder path. """

#     run(["mumax3-convert", "-png", f"simulations/{simulation_name}.out/*.ovf", "-o", "output"], stdout=PIPE, stderr=STDOUT)

# def display_folder(simulation_name):
#     """Displays all .png files in the given folder path."""

#     folder_path = f"simulations/{simulation_name}.out"

#     listOfImageNames = os.listdir(path=folder_path)
#     listOfImageNames = list(filter(lambda x:x[-4:]==".png", listOfImageNames))
#     listOfImageNames.sort()

#     for imageName in listOfImageNames:
#         display(Image(filename=os.path.join(folder_path, imageName)))

In [11]:
import numpy as np
import pickle

class MetaArray(np.ndarray):
    def __new__(cls, input_array, metadata=None):
        obj = np.asarray(input_array).view(cls)
        obj.metadata = metadata if metadata is not None else {}
        return obj

    # Need this function to initialize the array after the __new__
    def __array_finalize__(self, obj):
        if obj is None: return
        self.metadata = getattr(obj, 'metadata', None)

    def add_metadata(self, metadata: dict = {}):
        self.metadata = metadata

def save_figure_to_pickle(filename, fields, outputdir): 
    # Saves a dictonary to a pickle file
    with open(f"{outputdir}/{filename}.pkl", "wb") as f:
        pickle.dump(fields, f)

def load_figure_from_pickle(filename, outputdir):
    # Loads a dictonary with the different fields
    with open(f"{outputdir}/{filename}.pkl", "rb") as f:
        return pickle.load(f)

def convert_ovf_to_npy(simulation_dir)
    # Find all .ovf files in all .out folders
    all_ovf_files = glob(os.path.join(simulation_dir, "*.out", "*.ovf"))
    
    if all_ovf_files:
        # Run mumax3-convert once for all files
        p = run(["mumax3-convert", "-numpy", *all_ovf_files], stdout=PIPE, stderr=STDOUT)
        if p.returncode != 0:
            print("Error converting OVF files:")
            print(p.stdout.decode('UTF-8'))

def read_mumax3_ovffiles():
    """Load all ovffiles in outputdir into a dictionary of numpy arrays
    with the ovffilename (without extension) as key"""

    simulation_folder = "simulations"
    
    convert_ovf_to_npy(simulation_folder)

    fields = {}

    out_dirs = [d for d in glob(os.path.join(simulation_folder, "*.out"))]
    
    for out_dir in out_dirs:
        folder_name = os.path.basename(out_dir).replace(".out", "")
        
        # Load all .npy files in the folder (after conversion)
        npy_files = glob(os.path.join(out_dir, "*.npy"))
        if not npy_files:
            print(f"No numpy files found in {out_dir}")
            continue

        # Assuming each folder has a single OVF file
        npy_file = npy_files[0]
        arr = np.load(npy_file)[:, 0, :, :]  # mx, my, mz
        
        # Extract metadata from folder name, e.g., small_fig_50nm_1
        parts = folder_name.split("_")
        try:
            width = int(parts[2].replace("nm",""))
        except (IndexError, ValueError):
            width = None  # fallback if naming does not match
        pixel_size = width * 1e-9 if width else None
        image_width = arr.shape[0]

        meta_array = MetaArray(arr, {
            "width": width,
            "pixel_size": pixel_size,
            "image_width": image_width
        })

        fields[folder_name] = meta_array

    return fields

def read_mumax3_table(filename):
    """Puts the mumax3 output table in a pandas dataframe"""

    table = read_table(filename)
    table.columns = ' '.join(table.columns).split()[1::2]

    return table

def run_mumax3(script, name, verbose=False):
    """ Executes a mumax3 script and convert ovf files to numpy files

    Parameters
    ----------
      script:  string containing the mumax3 input script
      name:    name of the simulation (this will be the name of the script and output dir)
      verbose: print stdout of mumax3 when it is finished
    """
    if not os.path.exists("simulations"):
        os.mkdir("simulations")

    scriptfile = "simulations/" + name + ".txt"
    outputdir  = "simulations/" + name + ".out"

    # write the input script in scriptfile
    with open(scriptfile, 'w' ) as f:
        f.write(script)

    # call mumax3 to execute this script
    p = run(["mumax3","-f",scriptfile], stdout=PIPE, stderr=STDOUT)
    if verbose or p.returncode != 0:
        print(p.stdout.decode('UTF-8'))

    # if os.path.exists(outputdir + "/table.txt"):
    #     table = read_mumax3_table(outputdir + "/table.txt")
    # else:
    #     table = None

    # fields = read_mumax3_ovffiles(outputdir)

    # # return table, fields
    # return fields



In [ ]:
def make_simulation_script(figure_width, image_width, pixel_size, squirclenes):
    """
    Parameters are given in nm, the rest will be configured from there.
    """
    # SIMULATION_GRID_LENGTH = 128 
    # pixel_scaling = image_shape_in_pixels / SIMULATION_GRID_LENGTH
    # simulation_pixel_size = round((image_shape_in_pixels / SIMULATION_GRID_LENGTH) * pixel_size, 3)

    simulation_grid_width = image_width // pixel_size
    
    simulation_figure_width = figure_width // pixel_size

    if simulation_figure_width > simulation_grid_width:
        print("Image_width is larger than figure width, please swap these around..")
        return

    mumax_script = f"""
    setgridsize({simulation_grid_width}, {simulation_grid_width}, 1)
    setcellsize({pixel_size}e-9, {pixel_size}e-9, {pixel_size}e-9)
    
    // Magnetisation saturation
    msat = 8.6e5
    aex = 13e-12
    // "Friction constant", how fast it will reach equilibrium
    alpha = 0.5
    
    N := 6
    size := {simulation_figure_width}
    
    interop := {squirclenes}/(N-1)
    radius := size*interop/2
    circle_x_position := abs(size)/2 - radius
    circle_y_position := abs(size)/2 - radius
    cross_width := size - radius*2

    horisontal_rectangle := rect(size, cross_width)
    vertical_rectangle := rect(cross_width, size)
    edge_circle := circle(radius*2)
    circles := edge_circle.transl(circle_x_position, circle_y_position, 0).add(edge_circle.transl(-circle_x_position, circle_y_position, 0)).add(edge_circle.transl(-circle_x_position, -circle_y_position, 0)).add(edge_circle.transl(circle_x_position, -circle_y_position, 0))

    squircle := horisontal_rectangle.add(vertical_rectangle).add(circles)

    setgeom(squircle)
    m = randommag()
    m = vortex(1, 1)
    relax() // Obsolete when already minimized with vortex..?
    minimize()

    save(m)
    """
    return mumax_script


In [ ]:
# Run the simulations for all of the files.

# These numbers are given in nm sizes
pixel_widths_small = np.array([100, 130, 160, 190, 220, 250])
image_size_small = 300
pixel_size_small = 1

pixel_widths_large = np.array([200, 260, 320, 380, 440, 500])
image_size_large = 600
pixel_size_large = 2

for i, pixel_width in enumerate(pixel_widths_small):
    # Go through the pixel widths 
    for j in range(6):
        # Go through the different squircicle sizes, square->circle
        name = f"small_fig_{pixel_width}nm_{j+1}"
        mumax_script = make_simulation_script(
            image_size_small,
            pixel_width,
            pixel_size_small,
            j
        )
        run_mumax3(mumax_script, name)

for i, pixel_width in enumerate(pixel_widths_large):
    # Go through the pixel widths 
    for j in range(6):
        # Go through the different squircicle sizes, square->circle
        name = f"large_fig_{pixel_width}nm_{j+1}"
        mumax_script = make_simulation_script(
            image_size_large,
            pixel_width,
            pixel_size_large,
            j
        )
        run_mumax3(mumax_script, name)
